In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.serif'] = ['SimHei']

import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

# 航班数据
feature = pd.read_table('/Users/tuyu/000000毕业设计/Flight_Delay/Train_Feature/final_feature.csv',sep=',',encoding='gb2312')
feature.head()

,航班编号,计划起飞时间,计划到达时间,计划起飞时刻,计划到达时刻,计划飞行时间,起飞间隔,前序延误,平均延误时间,最大延误时间,...,本时段实际进港延误航班数,本时段离港延误时长,本时段离港平均延误时长,本时段离港延误率,上一时段离港平均延误时长,上一时段进港延误时长,上一时段离港延误时长,上一时段实际进港航班数,上一时段进港延误率,标签
0,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,CZ3855,16801,16801,0,3,3.666667,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,CZ3855,16801,16801,0,1,1.250000,NaN,NaN,1.963379,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,CZ3295,16801,16801,0,1,1.166667,NaN,NaN,0.324175,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [2]:
feature0 = feature[feature['标签'] == 0]
feature0new = feature0.sample(frac=0.1, replace=False, random_state=None,axis=0)
feature1 = feature[feature['标签'] == 1]
featureGBDT = pd.concat([feature0new,feature1],axis=0, ignore_index=True)
featureGBDT = featureGBDT.sample(frac=0.1, replace=False, random_state=None,axis=0)

In [6]:
feature0LSTM = feature0new.sample(frac=0.2, replace=False, random_state=None,axis=0)
feature1LSTM = feature1.sample(frac=0.2, replace=False, random_state=None,axis=0)
featureLSTM = pd.concat([feature0LSTM,feature1LSTM],axis=0, ignore_index=True)
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
del(featureLSTM['本时段离港平均延误时长'])
del(featureLSTM['上一时段离港平均延误时长'])
featureLSTM['起飞间隔'] = featureLSTM['起飞间隔'].fillna(0)
featureLSTM['前序延误'] = featureLSTM['前序延误'].fillna(0)
featureLSTM['出发机场天气得分'] = featureLSTM['出发机场天气得分'].interpolate()
featureLSTM['到达机场天气得分'] = featureLSTM['到达机场天气得分'].interpolate()
featureLSTM['平均延误时间'] = featureLSTM['平均延误时间'].interpolate()
featureLSTM['最大延误时间'] = featureLSTM['最大延误时间'].interpolate()
featureLSTM['延误时间中位数'] = featureLSTM['延误时间中位数'].interpolate()
featureLSTM['延误时间标准差'] = featureLSTM['延误时间标准差'].interpolate()
featureLSTM['本时段计划离港航班数'] = featureLSTM['本时段计划离港航班数'].fillna(featureLSTM['本时段计划离港航班数'].mean())
featureLSTM['本时段实际离港航班数'] = featureLSTM['本时段实际离港航班数'].fillna(featureLSTM['本时段实际离港航班数'].mean())
featureLSTM['本时段进港延误时长'] = featureLSTM['本时段进港延误时长'].fillna(featureLSTM['本时段进港延误时长'].mean())
featureLSTM['本时段实际进港延误航班数'] = featureLSTM['本时段实际进港延误航班数'].fillna(featureLSTM['本时段实际进港延误航班数'].mean())
featureLSTM['本时段离港延误时长'] = featureLSTM['本时段离港延误时长'].fillna(featureLSTM['本时段离港延误时长'].mean())
featureLSTM['本时段离港延误率'] = featureLSTM['本时段离港延误率'].fillna(featureLSTM['本时段离港延误率'].mean())
featureLSTM['上一时段进港延误时长'] = featureLSTM['上一时段进港延误时长'].fillna(featureLSTM['上一时段进港延误时长'].mean())
featureLSTM['上一时段离港延误时长'] = featureLSTM['上一时段离港延误时长'].fillna(featureLSTM['上一时段离港延误时长'].mean())
featureLSTM['上一时段实际进港航班数'] = featureLSTM['上一时段实际进港航班数'].fillna(featureLSTM['上一时段实际进港航班数'].mean())
featureLSTM['上一时段进港延误率'] = featureLSTM['上一时段进港延误率'].fillna(featureLSTM['上一时段进港延误率'].mean())



In [9]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd 
import numpy as np
from numpy import *
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

dataset = featureLSTM.as_matrix()
X = dataset[:,1:36]
Y = dataset[:,36]
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2, random_state=7)

x_train = (x_train - x_train.min(axis=0)) / (x_train.max(axis=0) - x_train.min(axis=0))
x_test = (x_test - x_test.min(axis=0)) / (x_test.max(axis=0) - x_test.min(axis=0))

/Users/tuyu/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


In [10]:
import pandas as pd
from keras.utils import to_categorical
from keras.layers import Dense,RNN,LSTM,Activation,Dropout
from keras.models import Sequential

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(len(x_train[0]), 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 161435 samples, validate on 40359 samples
Epoch 1/20
161435/161435 [==============================] - 153s 949us/step - loss: 0.6852 - acc: 0.5623 - val_loss: 0.6834 - val_acc: 0.5600
Epoch 2/20
161435/161435 [==============================] - 181s 1ms/step - loss: 0.6853 - acc: 0.5623 - val_loss: 0.6859 - val_acc: 0.5600
Epoch 3/20
161435/161435 [==============================] - 176s 1ms/step - loss: 0.6854 - acc: 0.5623 - val_loss: 0.6858 - val_acc: 0.5600
Epoch 4/20
161435/161435 [==============================] - 175s 1ms/step - loss: 0.6853 - acc: 0.5623 - val_loss: 0.6858 - val_acc: 0.5600
Epoch 5/20
161435/161435 [==============================] - 168s 1ms/step - loss: 0.6853 - acc: 0.5623 - val_loss: 0.6859 - val_acc: 0.5600
Epoch 6/20
161435/161435 [==============================] - 147s 909us/step - loss: 0.6763 - acc: 0.5

In [12]:
from sklearn import metrics

predict_result_train = model.predict_classes(x_train)
y_train = y_train.tolist()
y_train = np.array([i for i in y_train])

predict_result_train = predict_result_train.tolist()
predict_result_train = np.array(predict_result_train)

testAccuracy = metrics.accuracy_score(y_train, predict_result_train)
print("训练样本的准确率: %.4f" % testAccuracy)

x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

predict_result = model.predict_classes(x_test)
y_test= y_test.tolist()
y_test = np.array([i for i in y_test])

predict_result = predict_result.tolist()
predict_result = np.array(predict_result)

trainAccuracy = metrics.accuracy_score(y_test, predict_result)
print("测试样本的准确率: %.4f" % trainAccuracy)

201794/201794 [==============================] - 52s 256us/step
训练样本的准确率: 0.7909
50449/50449 [==============================] - 13s 264us/step
测试样本的准确率: 0.5410


In [13]:
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()

print(ks(predict_result,y_test))

from sklearn.metrics import classification_report 
print(classification_report(predict_result, y_test))

('ks', 0.15682724478509913)
              precision    recall  f1-score   support

           0       0.28      0.75      0.40     10437
           1       0.88      0.49      0.63     40012

    accuracy                           0.54     50449
   macro avg       0.58      0.62      0.52     50449
weighted avg       0.76      0.54      0.58     50449



In [14]:
import pandas as pd
from keras.utils import to_categorical
from keras.layers import Dense,RNN,LSTM,Activation,Dropout
from keras.models import Sequential


x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
model = Sequential()
model.add(LSTM(units=30, activation='relu', return_sequences=True, input_shape=(len(x_train[0]), 1)))
model.add(Dropout(0.1))
model.add(LSTM(units=30, activation='relu', return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=64, validation_split = 0.2, epochs=20, shuffle=False, verbose=1)

Train on 161435 samples, validate on 40359 samples
Epoch 1/20
161435/161435 [==============================] - 114s 708us/step - loss: 0.6704 - acc: 0.5838 - val_loss: 0.5683 - val_acc: 0.7107
Epoch 2/20
161435/161435 [==============================] - 116s 720us/step - loss: 0.5479 - acc: 0.7301 - val_loss: 0.5134 - val_acc: 0.7507
Epoch 3/20
161435/161435 [==============================] - 133s 825us/step - loss: 0.5176 - acc: 0.7510 - val_loss: 0.4935 - val_acc: 0.7632
Epoch 4/20
161435/161435 [==============================] - 152s 943us/step - loss: 0.5027 - acc: 0.7617 - val_loss: 0.4842 - val_acc: 0.7710
Epoch 5/20
161435/161435 [==============================] - 135s 835us/step - loss: 0.4945 - acc: 0.7675 - val_loss: 0.4785 - val_acc: 0.7749
Epoch 6/20
161435/161435 [==============================] - 116s 722us/step - loss: 0.4892 - acc: 0.7703 - val_loss: 0.4739 - val_acc: 0.7774
Epoch 7/20
161435/161435 [==============================] - 122s 756us/step - loss: 0.4835 - acc:

In [15]:
from sklearn import metrics

predict_result_train = model.predict_classes(x_train)
y_train = y_train.tolist()
y_train = np.array([i for i in y_train])

predict_result_train = predict_result_train.tolist()
predict_result_train = np.array(predict_result_train)

testAccuracy = metrics.accuracy_score(y_train, predict_result_train)
print("训练样本的准确率: %.4f" % testAccuracy)

x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

predict_result = model.predict_classes(x_test)
y_test= y_test.tolist()
y_test = np.array([i for i in y_test])

predict_result = predict_result.tolist()
predict_result = np.array(predict_result)

trainAccuracy = metrics.accuracy_score(y_test, predict_result)
print("测试样本的准确率: %.4f" % trainAccuracy)

201794/201794 [==============================] - 48s 240us/step
训练样本的准确率: 0.8010
50449/50449 [==============================] - 12s 240us/step
测试样本的准确率: 0.4419


In [16]:
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()

print(ks(predict_result,y_test))

from sklearn.metrics import classification_report 
print(classification_report(predict_result, y_test))

('ks', 0.004667383491827604)
              precision    recall  f1-score   support

           0       0.01      0.67      0.03       559
           1       0.99      0.44      0.61     49890

    accuracy                           0.44     50449
   macro avg       0.50      0.55      0.32     50449
weighted avg       0.98      0.44      0.60     50449

